In [ ]:
%%capture
%pip install openai==0.27.0

In [ ]:
import openai
import random
import pandas as pd

# model: gpt-3.5-turbo or gpt-4
model_name = "gpt-3.5-turbo"

In [ ]:
try:
  api_key = "key" # Add API key
  openai.api_key = api_key
  print("API key successfully loaded.")
except:
  print("No API key file found.")

API key successfully loaded.


In [ ]:
# CSV path (change accordingly)
csv_path = "exp1_test.csv"
prompts = []
fields = ['Sentence', 'Type', 'P', 'Not P', 'Contradiction', 'Connective']  # Columns used in CSV
row_order = 1

# Reading CSV to dataframe
prompts_df = pd.read_csv(csv_path, usecols=fields)
for _, row in prompts_df.iterrows():
    prompt = {}
    prompt["connective"] = row["Connective"]
    prompt["sentence"] = row["Sentence"]
    prompt["contradiction"] = row["Contradiction"]
    prompt["type"] = row["Type"]
    prompt["order"] = row_order
    row_order += 1
    prompts.append(prompt)

print(f"Found {len(prompts)} prompts in the given CSV file.")

FileNotFoundError: [Errno 2] No such file or directory: 'lungu_scale_updated_ac.csv'

In [ ]:
results = []
row_order = 0

instructions_dict = {
    1: "Is the following sentence a contradiction?", # Contradiction 1 and output yes is good
    2: "Does the following sentence make sense?", # Contradiciton 1 and output no is good
    3: "Is the following sentence acceptable?", # Contradiciton 1 and output no is good
    4: "Is the following sentence natural?", # Contradiciton 1 and output no is good
}

# Calling GPT and appending responses to outputs list
for key, value in instructions_dict.items():
    for prompt in prompts:
        stimuli = f"{value} {prompt["sentence"]}"
        output = openai.ChatCompletion.create(
                      messages=[
                        {"role": "system", "content": "Respond with only 'yes' or 'no'"},
                        {"role": "user", "content": stimuli},
                      ]
                    )
        response = output['choices'][0]['message']['content']
        row_order += 1
        results.append({
            "Sentence": prompt["sentence"],
            "Connective": prompt["connective"],
            "Contradiction": prompt["contradiction"],
            "Instructions": key,
            "Type": prompt["type"],
            "Original order": prompt["order"],
            "Total order": row_order,
            "Response": response
        })

        print(stimuli)
        print(response)

        total_tokens += response["usage"]["total_tokens"]

print(f"Messages generated with {total_tokens} tokens")

In [ ]:
# Create dataframe from results
df = pd.DataFrame(results)

# Save the dataframe to a CSV file
output_csv_path = "output_responses_GPT_3.5_prompting_exp1_test.csv"
df.to_csv(output_csv_path, index=False)

print(f"Responses saved to {output_csv_path}")

Responses saved to output_responses_5.csv


In [ ]:
# For easier testing without needing to create a new CSV, but doesn't result in CSV output
# Set of examples
statements = [
    "The man draws the image although the man does not draw the image.",
    "The tree in my backyard has green leaves if the tree in my backyard does not have green leaves.",
    "The man draws the image but the man does not draw the image.",
    "Jim might have showed the man his tickets, Jim might not have showed the man his tickets",
    "The table could be round, the table could be not round."
]

outputs = []
instructions = "Is the following statement a contradiction?"

# Calling chatGPT and appending responses to outputs list
for statement in statements:
  gpt_message = openai.ChatCompletion.create(
                stimuli = f"{instructions} {statement}"
                messages=[
                  {"role": "system", "content": "Respond with only 'yes' or 'no'"},
                  {"role": "user", "content": stimuli},
                ]
              )
  respnse = gpt_message['choices'][0]['message']['content']
  outputs.append(response)

  total_tokens += response["usage"]["total_tokens"]
  print(statement)
  print(response)

print(f"Messages generated with {total_tokens} tokens")

Messages generated with 0 tokens
